# Preparação do ambiente

## Bibliotecas

In [1]:
import configparser
import hashlib
import json
import os
import pandas as pd
import requests
import time

from pathlib import Path
from tqdm.auto import tqdm

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

## Constantes e funções auxiliares

In [3]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'

In [4]:
def get_new_access_token(actual_refresh_token,client_id,client_secret):
    url = 'https://api.mercadolibre.com/oauth/token'
    payload = {
        'grant_type': 'refresh_token', 
        'client_id': client_id,
        'client_secret': client_secret, 
        'refresh_token': actual_refresh_token
    }
        
    headers = {
        'accept': 'application/json',
        'content-type': 'application/x-www-form-urlencoded'
    }
        
    response = requests.post(url, headers=headers, data=payload)
    content = response.content.decode(response.encoding)
    content =  json.loads(content)
        
    return content['access_token'], content['refresh_token']

In [5]:
def load_sch(sch_database_file):
    
    # load SCH database
    usecols = [0,1,11,12,13,14,15]
    dtype = {'Número de Homologação': 'str'}
    parse_dates = [0]
    date_format = '%d/%m/%Y'

    df_sch = pd.read_csv(
        sch_database_file,
        sep=';',
        usecols=usecols,
        dtype=dtype,
        parse_dates=parse_dates,
        date_format=date_format
        )

    df_sch = df_sch.sort_values(by='Data da Homologação',ascending=False)

    df_modelo = df_sch[['Número de Homologação','Modelo']].dropna()
    df_modelo = df_modelo.groupby('Número de Homologação',as_index=False)['Modelo'].apply(lambda x: ' | '.join(x))
    df_modelo = df_modelo.drop_duplicates(subset='Número de Homologação')

    df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].dropna()
    df_nome_comercial = df_nome_comercial.groupby('Número de Homologação',as_index=False)['Nome Comercial'].apply(lambda x: ' | '.join(x))
    df_nome_comercial = df_nome_comercial.drop_duplicates(subset='Número de Homologação')

    columns_to_keep = ['Data da Homologação', 'Número de Homologação', 'Nome do Fabricante']
    df_sch_models = df_sch[columns_to_keep]
    df_sch_models = df_sch_models.drop_duplicates(subset='Número de Homologação')

    df_sch_models = df_sch_models.merge(df_modelo,how='left')
    df_sch_models = df_sch_models.merge(df_nome_comercial,how='left')
    df_sch_models = df_sch.fillna('')
    df_sch_models['Modelo Completo'] = df_sch_models[['Modelo', 'Nome Comercial']].apply(lambda row: ' | '.join(row) if len(row['Nome Comercial'])>0 else row['Modelo'],axis=1)

    columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Modelo Completo']
    df_sch_models = df_sch_models.drop_duplicates(subset='Número de Homologação')
    df_sch_models = df_sch_models[columns_to_keep].reset_index(drop=True)
    
    return df_sch, df_sch_models

In [6]:
def parse_results(result, return_anatel_homologation_number=False):
    keys_to_keep = ['id', 'title', 'catalog_product_id', 'permalink', 'category_id', 'domain_id', 
                    'currency_id', 'price', 'original_price', 
                    'initial_quantity', 'available_quantity', 'official_store_id', 'official_store_name']
    
    parsed_result = {}
    for key in keys_to_keep:
        parsed_result[key] = result.get(key)
    
    if result.get('seller') is not None:
        parsed_result['seller_id'] = result.get('seller').get('id')
        parsed_result['seller_nickname'] = result.get('seller').get('nickname')
    else:
        parsed_result['seller_id'] = None
        parsed_result['seller_nickname'] = None    

    brand = None
    model = None
    gtin = None
    detailed_model = None
    anatel_homologation_number = None
    cellphones_anatel_homologation_number = None
    
    for attribute in result['attributes']:
        if attribute['id'] == 'BRAND':
            brand = attribute['value_name']
        elif attribute['id'] == 'MODEL':
            model = attribute['value_name']
        elif attribute['id'] == 'DETAILED_MODEL':
            detailed_model = attribute['value_name'] 
        elif attribute['id'] == 'GTIN':
            gtin = attribute['value_name'] 
        elif attribute['id'] == 'ANATEL_HOMOLOGATION_NUMBER':
            anatel_homologation_number = attribute['value_name']
        elif attribute['id'] == 'CELLPHONES_ANATEL_HOMOLOGATION_NUMBER':
            cellphones_anatel_homologation_number = attribute['value_name']

    if return_anatel_homologation_number:
        if anatel_homologation_number is None:
            item_id = parsed_result['id']
            attr_url = f'https://api.mercadolibre.com/items/{item_id}?attributes=attributes&include_internal_attributes=true'
            headers = {'user-agent': USER_AGENT}
            attr_response = requests.get(attr_url,headers=headers)
            attr_content = attr_response.content.decode(attr_response.encoding)
            attr_content =  json.loads(attr_content)
            attr_content = attr_content['attributes']
            for attr in attr_content:
                if attr['id'] == 'ANATEL_HOMOLOGATION_NUMBER':
                    anatel_homologation_number = attr['value_name']
    
        if cellphones_anatel_homologation_number is None:
            item_id = parsed_result['id']
            attr_url = f'https://api.mercadolibre.com/items/{item_id}?attributes=attributes&include_internal_attributes=true'
            headers = {'user-agent': USER_AGENT}
            attr_response = requests.get(attr_url,headers=headers)
            attr_content = attr_response.content.decode(attr_response.encoding)
            attr_content =  json.loads(attr_content)
            attr_content = attr_content['attributes']
            for attr in attr_content:
                if attr['id'] == 'CELLPHONES_ANATEL_HOMOLOGATION_NUMBER':
                    cellphones_anatel_homologation_number = attr['value_name']

    parsed_result['brand'] = brand
    parsed_result['model'] = model
    parsed_result['detailed_model'] = detailed_model
    parsed_result['gtin'] = gtin
    parsed_result['anatel_homologation_number'] = anatel_homologation_number
    parsed_result['cellphones_anatel_homologation_number'] = cellphones_anatel_homologation_number
    
    return parsed_result

# Carga e prepação dos dados

In [7]:
sch_database_file = '../../certificacao-homologacao/schwebsearch/datasets/sch_database/produtos_certificados.zip'
df_sch, df_sch_models = load_sch(sch_database_file)
df_sch

,Data da Homologação,Número de Homologação,Nome do Fabricante,Modelo,Nome Comercial,Categoria do Produto,Tipo do Produto
136048,2024-05-20,030572416551,Decathlon,8605113,W900,2,Transceptor de Radiação Restrita
138597,2024-05-20,004732404809,Elsys Equipamentos Eletrônicos Ltda,ESF-DE5100I,NaN,2,Sistemas de Identificação por Radiofrequências
137797,2024-05-20,061542414894,"Shenzhen Baseus Technology Co., Ltd.",PPAP2-10A,NaN,1,Acessório p/ Telefone Móvel Celular do tipo Ba...
140046,2024-05-20,050542403757,Lear Corporation,KOBJXF23A,NaN,2,Sistemas Operando nas Faixas de RF Ultra Larga
137799,2024-05-20,062092408867,"Fortinet, Inc.",FG-120G,NaN,3,Equipamento de Rede de Dados
...,...,...,...,...,...,...,...
378,2001-09-21,020050101504,Nortel Networks Inc.,Metrocell / 800 MFRM,Metrocell / 800 MFRM,3,Transceptor para Estação Rádio Base
332,2001-09-21,020060101504,Nortel Networks Inc.,Minicell / 800 MFRM,Minicell / 800 MFRM,3,Transceptor para Estação Rádio Base
345,2001-09-19,020020100563,Nokia do Brasil Tecnologia Ltda.,3320,3320,1,Telefone Móvel Celular
409,2001-09-19,020030100563,Nokia do Brasil Tecnologia Ltda.,Freedom TD1000,Freedom TD1000,1,Telefone Móvel Celular


In [8]:
df_sch_models

,Número de Homologação,Nome do Fabricante,Modelo Completo
0,030572416551,Decathlon,8605113 | W900
1,004732404809,Elsys Equipamentos Eletrônicos Ltda,ESF-DE5100I
2,061542414894,"Shenzhen Baseus Technology Co., Ltd.",PPAP2-10A
3,050542403757,Lear Corporation,KOBJXF23A
4,062092408867,"Fortinet, Inc.",FG-120G
...,...,...,...
69916,020050101504,Nortel Networks Inc.,Metrocell / 800 MFRM | Metrocell / 800 MFRM
69917,020060101504,Nortel Networks Inc.,Minicell / 800 MFRM | Minicell / 800 MFRM
69918,020020100563,Nokia do Brasil Tecnologia Ltda.,3320 | 3320
69919,020030100563,Nokia do Brasil Tecnologia Ltda.,Freedom TD1000 | Freedom TD1000


In [9]:
file_ean_celulares = '../datasets/lista_celulares_homologados_ean.xlsx'

dtype = {'Número de Homologação': 'str', 'Código EAN': 'str'}

df_ean_celulares = pd.read_excel(file_ean_celulares,dtype=dtype)
df_ean_celulares
df_ean_celulares = df_ean_celulares[df_ean_celulares['Código EAN'].str.len()>=13]
df_ean_celulares[df_ean_celulares['Código EAN'].str.len()>=13]
df_ean_celulares['Código EAN'] = df_ean_celulares['Código EAN'].apply(lambda x:  x.strip())

columns_to_keep = ['Código EAN', 'Número de Homologação']
df_ean_sch = df_ean_celulares[columns_to_keep].drop_duplicates()
# df_ean_sch = df_ean_sch.dropna().reset_index(drop=True)
# df_ean_sch = df_ean_sch.astype('int64')

df_ean_sch.columns = ['ean_sch', 'sch_sch']
df_ean_sch.head()

,ean_sch,sch_sch
0,7892597349623,022972000330
1,7892597349630,022972000330
2,7892597350971,013692100330
3,7892597350988,013692100330
4,7892597351749,013692100330


In [10]:
creds_file = Path(os.environ['USERPROFILE'],'creds.ini')
creds = configparser.ConfigParser()
creds.read(creds_file)  

client_id = creds['MERCADO_LIVRE']['client_id']
client_secret = creds['MERCADO_LIVRE']['client_secret']
actual_refresh_token = creds['MERCADO_LIVRE']['actual_refresh_token']
actual_access_token = creds['MERCADO_LIVRE']['actual_access_token']

In [11]:
def update_access_token():

    global actual_access_token
    global actual_refresh_token
    
    actual_access_token, actual_refresh_token = get_new_access_token(actual_refresh_token,client_id,client_secret)
    
    creds['MERCADO_LIVRE']['actual_access_token'] = actual_access_token
    creds['MERCADO_LIVRE']['actual_refresh_token'] = actual_refresh_token
    
    with open(creds_file, 'w') as file:
        creds.write(file)

# Análise

In [12]:
update_access_token()

In [14]:
url = 'https://api.mercadolibre.com/sites/MLB/search?category=MLB1055'
params = {'offset': 0}
headers = {
  'Authorization': f'Bearer {actual_access_token}'
}

response = requests.request("GET", url, headers=headers, params=params)
content = response.content.decode(response.encoding)
content =  json.loads(content)
results = content['results']

total_items = content['paging']['total']
next_offsets = list(range(50,4000,50))

In [15]:
for offset in tqdm(next_offsets):
    
    params = {'offset': offset}
    response = requests.request("GET", url, headers=headers, params=params)   
    
    if response.status_code == 200:
        content = response.content.decode(response.encoding)
        content =  json.loads(content)
        results.extend(content['results'])

  0%|          | 0/79 [00:00<?, ?it/s]

In [16]:
parsed_results = [parse_results(result) for result in tqdm(results)]

  0%|          | 0/3897 [00:00<?, ?it/s]

In [17]:
df_cellphones = pd.DataFrame(parsed_results)
df_cellphones.to_parquet('cellphone.parquet')

columns_to_keep = ['id', 'permalink', 'title','seller_nickname', 'brand', 'model', 'gtin', 'cellphones_anatel_homologation_number']
df_analise = df_cellphones[columns_to_keep]

columns_to_keep = ['id', 'permalink', 'title', 'seller_nickname', 'brand', 'model', 'ean_anuncio', 'sch_anuncio']
df_analise.columns = columns_to_keep

df_analise['ean_anuncio'] = df_analise['ean_anuncio'].str.split(',')
df_analise = df_analise.explode('ean_anuncio')

# df_analise['ean_presente'] = df_analise['ean_mp'].apply(lambda x:  0 if x is None else 1)
df_analise['sch_presente'] = df_analise['sch_anuncio'].apply(lambda x:  0 if x is None else 1)

df_analise['sch_anuncio'] = df_analise['sch_anuncio'].fillna('0').str.zfill(12)
df_analise['ean_anuncio'] = df_analise['ean_anuncio'].fillna('0').str.zfill(13)

df_analise = df_analise.merge(df_ean_sch,left_on='ean_anuncio', right_on='ean_sch', how='left')
df_analise['ean_ok'] = df_analise['ean_anuncio']==df_analise['ean_sch']
df_analise['sch_ok'] = df_analise['sch_anuncio']==df_analise['sch_sch']
df_analise['ean_sch_ok'] = df_analise[['ean_ok', 'sch_ok']].apply(lambda row: all(row),axis=1)
columns_to_keep = ['id', 'permalink', 'title', 'seller_nickname', 'brand', 'model', 'ean_anuncio', 'sch_anuncio', 'sch_presente', 'ean_sch', 'sch_sch', 'ean_sch_ok']
df_analise = df_analise[columns_to_keep]


df_analise = df_analise.merge(df_sch_models, left_on='sch_anuncio', right_on='Número de Homologação',how='left')
df_analise['sch_valido'] = df_analise['Número de Homologação'].apply(lambda x:  0 if pd.isna(x) else 1)
columns_to_keep = ['id', 'permalink', 'title', 'seller_nickname', 'brand', 'model', 'ean_anuncio', 'sch_anuncio', 'sch_presente', 'ean_sch', 'sch_sch', 'ean_sch_ok', 'sch_valido', 'Nome do Fabricante', 'Modelo Completo']
df_analise = df_analise[columns_to_keep]

df_analise

,id,permalink,title,seller_nickname,brand,model,ean_anuncio,sch_anuncio,sch_presente,ean_sch,sch_sch,ean_sch_ok,sch_valido,Nome do Fabricante,Modelo Completo
0,MLB3589269609,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 4g Dual Sim 128 Gb Azul Esc...,MERCADOLIVRE ELETRONICOS,Samsung,A15 4G,7892509134262,198252300953,1,7892509134262,198252300953,True,1,Samsung Electronics Co Ltd.,SM-A155M/DSN
1,MLB3683512465,https://www.mercadolivre.com.br/motorola-moto-...,Motorola Moto G04s 128gb Cinza 4gb Ram,MERCADOLIVRE ELETRONICOS,Motorola,G04S,7892597354160,032192400330,1,7892597354160,032192400330,True,1,Motorola Mobility LLC,XT2421-6
2,MLB4431810396,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 5g Tela De 6.5 90hz 256gb ...,MERCADOLIVRE ELETRONICOS,Samsung,A15 5G Dual Sim,7892509134675,198242300953,1,7892509134675,198242300953,True,1,Samsung Electronics Co Ltd.,SM-A156M/DSN
3,MLB4484849730,https://www.mercadolivre.com.br/motorola-moto-...,Motorola Moto G24 128gb Grafite 4gb Ram,MERCADOLIVRE ELETRONICOS,Motorola,G24,7892597353811,206672300330,1,7892597353811,206672300330,True,1,Motorola Mobility LLC,XT2423-6
4,MLB3589186955,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 Dual Sim 4g 256gb Azul Clar...,MERCADOLIVRE ELETRONICOS,Samsung,A15 4G,7892509135351,198252300953,1,7892509135351,198252300953,True,1,Samsung Electronics Co Ltd.,SM-A155M/DSN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3920,MLB4831338534,https://produto.mercadolivre.com.br/MLB-483133...,Samsung Galaxy M30s 64gb,OUTLET.DO.CELULAR,Samsung,M30s,0000000000000,000000000000,0,NaN,NaN,False,0,NaN,NaN
3921,MLB3683831365,https://produto.mercadolivre.com.br/MLB-368383...,Samsung Galaxy M52 6gb Ram Preto 128gb,FIXCOMM,Samsung,M52,0000000000000,000000000000,0,NaN,NaN,False,0,NaN,NaN
3922,MLB3758756755,https://produto.mercadolivre.com.br/MLB-375875...,Xiaomi Redmi 13 Dual Sim 256 Gb 8gb De Ram (la...,MARINHO7VENDAS,Xiaomi,13,0000000000000,000000000000,0,NaN,NaN,False,0,NaN,NaN
3923,MLB3761816315,https://produto.mercadolivre.com.br/MLB-376181...,Smartphone Xiaomi Redmi 13 (256gb / 8gb De Ram...,MARINHO7VENDAS,Xiaomi,13,0000000000000,000000000000,0,NaN,NaN,False,0,NaN,NaN


In [23]:
df_analise[df_analise['ean_anuncio']=='0000000000000'].shape

(2701, 15)

In [18]:
columns_to_group = ['id', 'sch_presente', 'sch_valido', 'ean_sch_ok']

agg_func = {'id': 'count', 'sch_presente': 'sum', 'sch_valido': 'sum', 'ean_sch_ok': 'sum'}
df_resumo = pd.DataFrame(df_analise[columns_to_group].agg(agg_func)).T

df_resumo['pe_sch_presente'] = df_resumo['sch_presente']/df_resumo['id']*100
df_resumo['pe_sch_valido'] = df_resumo['sch_valido']/df_resumo['id']*100
df_resumo['pe_ean_sch_ok'] = df_resumo['ean_sch_ok']/df_resumo['id']*100

columns_to_keep = ['Total de anúncios', 'Anúncios com código SCH Informado', 'Anúncios com código SCH Válido', 'Anúncios com par EAN/SCH Válido',
                   '% Anúncios com código SCH Informado', '% Anúncios com código SCH Válido', '% Anúncios com par EAN/SCH Válido']

df_resumo.columns = columns_to_keep
df_resumo.index.name = 'Mercado Livre'

df_resumo

,Total de anúncios,Anúncios com código SCH Informado,Anúncios com código SCH Válido,Anúncios com par EAN/SCH Válido,% Anúncios com código SCH Informado,% Anúncios com código SCH Válido,% Anúncios com par EAN/SCH Válido
Mercado Livre,,,,,,,
0,3925,2730,2716,280,69.55414,69.197452,7.133758


In [19]:
columns_to_group = ['seller_nickname', 'id', 'sch_presente', 'sch_valido', 'ean_sch_ok']
agg_func = {'id': 'count', 'sch_presente': 'sum', 'sch_valido': 'sum', 'ean_sch_ok': 'sum'}
df_seller = df_analise[columns_to_group].groupby(columns_to_group[0]).agg(agg_func)

df_seller['pe_sch_presente'] = df_seller['sch_presente']/df_seller['id']*100
df_seller['pe_sch_valido'] = df_seller['sch_valido']/df_seller['id']*100
df_seller['pe_ean_sch_ok'] = df_seller['ean_sch_ok']/df_seller['id']*100

columns_to_keep = ['Total de anúncios', 'Anúncios com EAN Informado', 'Anúncios com código SCH Informado', 'Anúncios com código SCH Válido', 
                   '% Anúncios com EAN Informado', '% Anúncios com código SCH Informado', '% Anúncios com código SCH Válido']

df_seller.columns = columns_to_keep
df_seller.index.name = 'Vendedor'

df_seller.sort_values(by='Total de anúncios',ascending=False).head(10)

,Total de anúncios,Anúncios com EAN Informado,Anúncios com código SCH Informado,Anúncios com código SCH Válido,% Anúncios com EAN Informado,% Anúncios com código SCH Informado,% Anúncios com código SCH Válido
Vendedor,,,,,,,
BPSTORE,148,116,115,0,78.378378,77.702703,0.000000
TROCAFONECOMERCIALIZACAO,84,52,52,5,61.904762,61.904762,5.952381
MERCADOLIVRE ELETRONICOS,70,70,70,69,100.000000,100.000000,98.571429
CELLFONE1,62,49,48,5,79.032258,77.419355,8.064516
ARIANERS8,52,44,44,0,84.615385,84.615385,0.000000
EDIMAX INFORMATICA,45,12,12,0,26.666667,26.666667,0.000000
IRANICEGAMA,43,28,28,0,65.116279,65.116279,0.000000
CELLTRONICS BRASIL,40,37,37,0,92.500000,92.500000,0.000000
ELLETECH,38,31,31,0,81.578947,81.578947,0.000000


In [20]:
columns_to_group = ['brand', 'id', 'sch_presente', 'sch_valido', 'ean_sch_ok']
agg_func = {'id': 'count', 'sch_presente': 'sum', 'sch_valido': 'sum', 'ean_sch_ok': 'sum'}
df_brand = df_analise[columns_to_group].groupby(columns_to_group[0]).agg(agg_func)

df_brand['pe_sch_presente'] = df_brand['sch_presente']/df_brand['id']*100
df_brand['pe_sch_valido'] = df_brand['sch_valido']/df_brand['id']*100
df_brand['pe_ean_sch_ok'] = df_brand['ean_sch_ok']/df_brand['id']*100

columns_to_keep = ['Total de anúncios', 'Anúncios com EAN Informado', 'Anúncios com código SCH Informado', 'Anúncios com código SCH Válido', 
                   '% Anúncios com EAN Informado', '% Anúncios com código SCH Informado', '% Anúncios com código SCH Válido']

df_brand.columns = columns_to_keep
df_brand.index.name = 'Fabricante'

df_brand.sort_values(by='Total de anúncios',ascending=False).head(10)

,Total de anúncios,Anúncios com EAN Informado,Anúncios com código SCH Informado,Anúncios com código SCH Válido,% Anúncios com EAN Informado,% Anúncios com código SCH Informado,% Anúncios com código SCH Válido
Fabricante,,,,,,,
Xiaomi,815,659,659,2,80.858896,80.858896,0.245399
Samsung,693,559,559,127,80.663781,80.663781,18.326118
Motorola,439,341,338,42,77.676538,76.993166,9.567198
Apple,278,243,243,63,87.410072,87.410072,22.661871
Multilaser,219,176,176,8,80.365297,80.365297,3.652968
LG,177,117,117,0,66.101695,66.101695,0.000000
Nokia,176,76,76,8,43.181818,43.181818,4.545455
Positivo,127,99,99,10,77.952756,77.952756,7.874016
Realme,95,84,81,0,88.421053,85.263158,0.000000
